In [1]:
import matchzoo as mz
from matchzoo import DataGenerator
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import scipy

from sklearn.metrics import precision_recall_fscore_support

Using TensorFlow backend.


# Load Data and Preprocess It

In [2]:
train_raw = mz.pack(pd.read_csv('toy_matchzoo_train.csv', index_col=0))
test_raw = mz.pack(pd.read_csv('toy_matchzoo_test.csv', index_col=0))

train_raw.relation['label'] = train_raw.relation['label'].astype('float32')
test_raw.relation['label'] = test_raw.relation['label'].astype('float32')

preprocessor = mz.preprocessors.DSSMPreprocessor()

preprocessor.fit(train_raw)
preprocessor.context

train_preprocessed = preprocessor.transform(train_raw)
test_preprocessed = preprocessor.transform(test_raw)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|█| 716/716 [00:00<00:00, 9927.94it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|█| 1025/1025 [00:00<00:00, 2980.44it/s]
Building VocabularyUnit from a datapack.: 100%|███████████████████████████| 112803/112803 [00:00<00:00, 2820918.39it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|█| 716/716 [00:00<00:00, 6476.73it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|█| 1025/1025 [00:00<00:00, 2653.31it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit 

# Build Model

In [3]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()

Parameter "name" set to DSSM.
Parameter "task" set to Ranking Task.


Since no loss function set, default is Mean Square Error

In [4]:
model.build()
model.compile()
print(model._params['task'].loss)


mse


# Train Model


In [5]:
x_train, y_train = train_preprocessed.unpack()
x_test, y_test = test_preprocessed.unpack()

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, verbose=2)

Train on 4986 samples, validate on 1248 samples
Epoch 1/20
 - 4s - loss: 0.1439 - val_loss: 0.1434
Epoch 2/20
 - 4s - loss: 0.1412 - val_loss: 0.1403
Epoch 3/20
 - 4s - loss: 0.1393 - val_loss: 0.1437
Epoch 4/20
 - 4s - loss: 0.1373 - val_loss: 0.1422
Epoch 5/20
 - 4s - loss: 0.1319 - val_loss: 0.1499
Epoch 6/20
 - 4s - loss: 0.1184 - val_loss: 0.1666
Epoch 7/20
 - 4s - loss: 0.0917 - val_loss: 0.1878
Epoch 8/20
 - 4s - loss: 0.0646 - val_loss: 0.2067
Epoch 9/20
 - 4s - loss: 0.0403 - val_loss: 0.2224
Epoch 10/20
 - 4s - loss: 0.0204 - val_loss: 0.2221
Epoch 11/20
 - 4s - loss: 0.0116 - val_loss: 0.2240
Epoch 12/20
 - 4s - loss: 0.0067 - val_loss: 0.2220
Epoch 13/20
 - 4s - loss: 0.0042 - val_loss: 0.2263
Epoch 14/20
 - 4s - loss: 0.0027 - val_loss: 0.2200
Epoch 15/20
 - 4s - loss: 0.0019 - val_loss: 0.2206
Epoch 16/20
 - 4s - loss: 0.0016 - val_loss: 0.2181
Epoch 17/20
 - 4s - loss: 0.0014 - val_loss: 0.2167
Epoch 18/20
 - 4s - loss: 0.0013 - val_loss: 0.2180
Epoch 19/20
 - 4s - loss:

# Evaluate

In [6]:
y_pred = model.predict(x_test)

print(y_pred[0:10])
print(y_test[0:10])





[[-0.00103   ]
 [ 0.03490742]
 [-0.00103   ]
 [ 0.02278818]
 [ 0.8897946 ]
 [ 0.14078303]
 [-0.00103   ]
 [ 0.03368205]
 [ 0.59130794]
 [ 0.05867494]]
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]]


In [7]:
#classify
y_pred_cutoff = y_pred > .5

prec, recall, f1beta, support = precision_recall_fscore_support(y_test, y_pred_cutoff, pos_label=1, average='binary')

print(prec)
print(recall)
print(f1beta)
print(f1_score(y_test, y_pred_cutoff, pos_label=1, average='binary'))

0.08943089430894309
0.052884615384615384
0.06646525679758308
0.06646525679758308
